In [33]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_mantine_components as dmc
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import webbrowser
from threading import Timer
import numpy as np

# Load the dataset
raw = 'dataset/backups/raw_data/public_emdat_20240923.xlsx'
file_path = 'dataset/cleaned_emrat.xlsx'
data = pd.read_excel(file_path)

In [16]:
df = pd.read_excel(raw, index_col = False)
df

,Unnamed: 0,id,type,iso,country,subregion,region,year,month,total_deaths,total_affected,total_damage,last_update,date,type_code
0,0,1999-9388-DJI,Drought,DJI,Djibouti,Sub-Saharan Africa,Africa,2001,6,NaN,100000.0,NaN,2023-09-25,2001/06,1
1,1,1999-9388-SDN,Drought,SDN,Sudan,Northern Africa,Africa,2000,1,NaN,2000000.0,NaN,2023-09-25,2000/01,1
2,2,1999-9388-SOM,Drought,SOM,Somalia,Sub-Saharan Africa,Africa,2000,1,21.0,1200000.0,NaN,2023-09-25,2000/01,1
3,3,2000-0002-AGO,Flood,AGO,Angola,Sub-Saharan Africa,Africa,2000,1,31.0,70000.0,17695.0,2023-09-25,2000/01,2
4,4,2000-0003-BGD,Extreme temperature,BGD,Bangladesh,Southern Asia,Asia,2000,1,49.0,NaN,NaN,2023-09-25,2000/01,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9204,10149,2024-9197-ZWE,Drought,ZWE,Zimbabwe,Sub-Saharan Africa,Africa,2024,4,NaN,7600000.0,NaN,2024-08-12,2024/04,1
9205,10150,2024-9277-VNM,Drought,VNM,Viet Nam,South-eastern Asia,Asia,2024,1,NaN,266000.0,NaN,2024-08-12,2024/01,1
9206,10151,2024-9338-NAM,Drought,NAM,Namibia,Sub-Saharan Africa,Africa,2024,5,NaN,1655000.0,NaN,2024-08-12,2024/05,1
9207,10152,2024-9381-GRD,Drought,GRD,Grenada,Latin America and the Caribbean,Americas,2024,5,NaN,100000.0,NaN,2024-08-12,2024/05,1


In [75]:
data['year'] = [int(y) for y in data['year']]

In [41]:
data['year'].dtype

dtype('int64')

In [79]:
ears = sorted(data['year'].unique())

In [82]:
disaster_types = sorted(data['type'].unique())
disaster_types

['Drought',
 'Earthquake',
 'Extreme temperature',
 'Flood',
 'Mass movement',
 'Storm',
 'Volcanic activity',
 'Wildfire']

In [7]:
min_year.dtype

dtype('int64')

In [1]:
import dash
from dash import dcc, html, no_update
from dash.dependencies import Input, Output, State
import dash_mantine_components as dmc
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go  
import webbrowser
from threading import Timer
# Load pre-defined functions that help our work
from utils import *

# Load the dataset
file_path = 'dataset/cleaned_emrat.xlsx'
data = pd.read_excel(file_path)


#### I. DATA & FUNCTIONS PREPARATION 
# Make sure datatypes are correct
data['last_update'] = pd.to_datetime(data['last_update'], errors='coerce')
# In dash, there is a problem with int64 dtype that'll not allow the dashboard to run.
# `year` is currently int64 and we work around this by converting `year` to str.
# When plotting with plotly, we'll convert it back to int later.
data['year'] = data['year'].astype(str)

# Calculate key statistics
## Total statistics
total_deaths = data['total_deaths'].sum()
total_affected = data['total_affected'].sum()
total_damage = data['total_damage'].sum()

## Most statistics
most_deaths_country = data.groupby('country')['total_deaths'].sum().idxmax()
most_affected_country = data.groupby('country')['total_affected'].sum().idxmax()
most_damaged_country = data.groupby('country')['total_damage'].sum().idxmax()

# Last update date
last_updated = data['last_update'].max()

# Prepare Year + Month filter data
data['YearMonth'] = data['year'].astype(str) + '-' + data['month'].astype(str)

# Get unique values for the dropdowns
years = sorted(data['year'].unique())
months = sorted(data['month'].unique())
continents = sorted(data['region'].unique())
subregions = sorted(data['subregion'].unique())
countries = sorted(data['country'].unique())
disaster_types = sorted(data['type'].unique())

# Get max-min year
max_year = max(years)
min_year = min(years)

In [8]:
agg_damage = data.groupby('country')['total_damage'].sum().reset_index()
agg_damage

,country,total_damage
0,Afghanistan,236461.0
1,Albania,928904.0
2,Algeria,10424748.0
3,American Samoa,324794.0
4,Angola,20181.0
...,...,...
212,Viet Nam,27634484.0
213,Wallis and Futuna Islands,0.0
214,Yemen,846750.0
215,Zambia,0.0


In [10]:
data['damage_by_country'] = agg_damage['total_damage']
data['damage_by_country']

0         236461.0
1         928904.0
2       10424748.0
3         324794.0
4          20181.0
           ...    
9204           NaN
9205           NaN
9206           NaN
9207           NaN
9208           NaN
Name: damage_by_country, Length: 9209, dtype: float64

In [12]:
dict = data.to_dict('Series')
dict

{'Unnamed: 0': 0          0
 1          1
 2          2
 3          3
 4          4
         ... 
 9204    9204
 9205    9205
 9206    9206
 9207    9207
 9208    9208
 Name: Unnamed: 0, Length: 9209, dtype: int64,
 'id': 0       1999-9388-DJI
 1       1999-9388-SDN
 2       1999-9388-SOM
 3       2000-0002-AGO
 4       2000-0003-BGD
             ...      
 9204    2024-9197-ZWE
 9205    2024-9277-VNM
 9206    2024-9338-NAM
 9207    2024-9381-GRD
 9208    2024-9592-AGO
 Name: id, Length: 9209, dtype: object,
 'type': 0                   Drought
 1                   Drought
 2                   Drought
 3                     Flood
 4       Extreme temperature
                ...         
 9204                Drought
 9205                Drought
 9206                Drought
 9207                Drought
 9208                Drought
 Name: type, Length: 9209, dtype: object,
 'iso': 0       DJI
 1       SDN
 2       SOM
 3       AGO
 4       BGD
        ... 
 9204    ZWE
 9205    VNM
 9206 

In [15]:
data

,Unnamed: 0,id,type,iso,country,subregion,region,year,month,total_deaths,total_affected,total_damage,last_update,date,type_code,total_damage_total_country,damage_category,YearMonth,damage_by_country
0,0,1999-9388-DJI,Drought,DJI,Djibouti,Sub-Saharan Africa,Africa,2001,6,NaN,100000.0,NaN,2023-09-25,2001/06,1,0,0 - 1M,2001-6,236461.0
1,1,1999-9388-SDN,Drought,SDN,Sudan,Northern Africa,Africa,2000,1,NaN,2000000.0,NaN,2023-09-25,2000/01,1,1062976,1M - 10M,2000-1,928904.0
2,2,1999-9388-SOM,Drought,SOM,Somalia,Sub-Saharan Africa,Africa,2000,1,21.0,1200000.0,NaN,2023-09-25,2000/01,1,1377654,1M - 10M,2000-1,10424748.0
3,3,2000-0002-AGO,Flood,AGO,Angola,Sub-Saharan Africa,Africa,2000,1,31.0,70000.0,17695.0,2023-09-25,2000/01,2,20181,0 - 1M,2000-1,324794.0
4,4,2000-0003-BGD,Extreme temperature,BGD,Bangladesh,Southern Asia,Asia,2000,1,49.0,NaN,NaN,2023-09-25,2000/01,3,14427631,10M - 100M,2000-1,20181.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9204,9204,2024-9197-ZWE,Drought,ZWE,Zimbabwe,Sub-Saharan Africa,Africa,2024,4,NaN,7600000.0,NaN,2024-08-12,2024/04,1,1382895,1M - 10M,2024-4,NaN
9205,9205,2024-9277-VNM,Drought,VNM,Viet Nam,South-eastern Asia,Asia,2024,1,NaN,266000.0,NaN,2024-08-12,2024/01,1,27634484,10M - 100M,2024-1,NaN
9206,9206,2024-9338-NAM,Drought,NAM,Namibia,Sub-Saharan Africa,Africa,2024,5,NaN,1655000.0,NaN,2024-08-12,2024/05,1,189931,0 - 1M,2024-5,NaN
9207,9207,2024-9381-GRD,Drought,GRD,Grenada,Latin America and the Caribbean,Americas,2024,5,NaN,100000.0,NaN,2024-08-12,2024/05,1,1434112,1M - 10M,2024-5,NaN


In [13]:
df = pd.DataFrame(dict, index = )
df

,Unnamed: 0,id,type,iso,country,subregion,region,year,month,total_deaths,total_affected,total_damage,last_update,date,type_code,total_damage_total_country,damage_category,YearMonth,damage_by_country
0,0,1999-9388-DJI,Drought,DJI,Djibouti,Sub-Saharan Africa,Africa,2001,6,NaN,100000.0,NaN,2023-09-25,2001/06,1,0,0 - 1M,2001-6,236461.0
1,1,1999-9388-SDN,Drought,SDN,Sudan,Northern Africa,Africa,2000,1,NaN,2000000.0,NaN,2023-09-25,2000/01,1,1062976,1M - 10M,2000-1,928904.0
2,2,1999-9388-SOM,Drought,SOM,Somalia,Sub-Saharan Africa,Africa,2000,1,21.0,1200000.0,NaN,2023-09-25,2000/01,1,1377654,1M - 10M,2000-1,10424748.0
3,3,2000-0002-AGO,Flood,AGO,Angola,Sub-Saharan Africa,Africa,2000,1,31.0,70000.0,17695.0,2023-09-25,2000/01,2,20181,0 - 1M,2000-1,324794.0
4,4,2000-0003-BGD,Extreme temperature,BGD,Bangladesh,Southern Asia,Asia,2000,1,49.0,NaN,NaN,2023-09-25,2000/01,3,14427631,10M - 100M,2000-1,20181.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9204,9204,2024-9197-ZWE,Drought,ZWE,Zimbabwe,Sub-Saharan Africa,Africa,2024,4,NaN,7600000.0,NaN,2024-08-12,2024/04,1,1382895,1M - 10M,2024-4,NaN
9205,9205,2024-9277-VNM,Drought,VNM,Viet Nam,South-eastern Asia,Asia,2024,1,NaN,266000.0,NaN,2024-08-12,2024/01,1,27634484,10M - 100M,2024-1,NaN
9206,9206,2024-9338-NAM,Drought,NAM,Namibia,Sub-Saharan Africa,Africa,2024,5,NaN,1655000.0,NaN,2024-08-12,2024/05,1,189931,0 - 1M,2024-5,NaN
9207,9207,2024-9381-GRD,Drought,GRD,Grenada,Latin America and the Caribbean,Americas,2024,5,NaN,100000.0,NaN,2024-08-12,2024/05,1,1434112,1M - 10M,2024-5,NaN
